In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv(rf"D:\Downloads\orders.csv")
df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian
...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian


In [4]:
users = pd.read_json(rf"D:\Downloads\users.json")
users

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold
...,...,...,...,...
2995,2996,User_2996,Hyderabad,Gold
2996,2997,User_2997,Hyderabad,Regular
2997,2998,User_2998,Bangalore,Regular
2998,2999,User_2999,Pune,Regular


In [5]:
rest = pd.read_excel(r'D:\Downloads\BOX1.csv.xlsx')

In [6]:
rest

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8
...,...,...,...,...
495,496,Restaurant_496,Indian,3.1
496,497,Restaurant_497,Mexican,4.4
497,498,Restaurant_498,Chinese,3.9
498,499,Restaurant_499,Mexican,4.9


In [7]:
order_users = df.merge(users,on='user_id',how='left')

In [8]:
df = order_users.merge(rest,on='restaurant_id',how='left',suffixes=('_order', '_master'))

In [9]:
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_order,name,city,membership,restaurant_name_master,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [10]:
df[df["membership"] == "Gold"]["total_amount"]

2        163.93
3       1155.97
8        953.30
11      1484.65
13       898.24
         ...   
9993     322.54
9994     137.96
9995    1211.96
9997     979.44
9998    1105.93
Name: total_amount, Length: 4987, dtype: float64

In [11]:
gold_city_revenue = (df[df["membership"] == "Gold"].groupby("city")["total_amount"].sum().sort_values(ascending=False))
gold_city_revenue.head(1)

city
Chennai    1080909.79
Name: total_amount, dtype: float64

In [12]:
cuisine_aov = (df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False))
cuisine_aov.head(1)

cuisine
Mexican    808.021344
Name: total_amount, dtype: float64

In [14]:
high_value_users = (df.groupby("user_id")["total_amount"].sum().reset_index())


In [15]:
count_users = high_value_users[high_value_users["total_amount"] > 1000]["user_id"].nunique()
count_users

2544

In [16]:
rating_bins = [0, 3.5, 4.0, 4.5, 5.0]
rating_labels = ["<3.5", "3.5–4.0", "4.0–4.5", "4.5–5.0"]

df["rating_range"] = pd.cut(df["rating"],bins=rating_bins,labels=rating_labels,include_lowest=True)

In [17]:
rating_revenue = (df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False))
rating_revenue

C:\Users\jakkoju vikasraj\AppData\Local\Temp\ipykernel_15756\2623278626.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


rating_range
4.5–5.0    2197030.75
<3.5       2136772.70
4.0–4.5    1960326.26
3.5–4.0    1717494.41
Name: total_amount, dtype: float64

In [18]:
top_rating_range = rating_revenue.idxmax()
top_rating_range

'4.5–5.0'

In [19]:
gold_city_aov = (df[df["membership"] == "Gold"].groupby("city")["total_amount"].mean().sort_values(ascending=False))
gold_city_aov

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [22]:
cuisine_stats = (df.groupby("cuisine").agg(distinct_restaurants=("restaurant_id", "nunique"),total_revenue=("total_amount", "sum")))

In [23]:
cuisine_stats.sort_values(by=["distinct_restaurants", "total_revenue"],ascending=[True, False]
)

,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [24]:
cuisine_stats.sort_values( by=["distinct_restaurants", "total_revenue"], ascending=[True, False]).head(1)

,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65


In [25]:
gold_orders = df[df["membership"] == "Gold"]["order_id"].nunique()
total_orders = df["order_id"].nunique()


percentage_gold_orders = round((gold_orders / total_orders) * 100)

percentage_gold_orders

50

In [44]:
restaurant_stats = (df.groupby(["restaurant_id", "restaurant_name_master", "restaurant_name_order"]).agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("order_id", "nunique")))



In [45]:
filtered_restaurants = restaurant_stats[restaurant_stats["total_orders"] < 20]

In [46]:
filtered_restaurants

,,,avg_order_value,total_orders
restaurant_id,restaurant_name_master,restaurant_name_order,,
1,Restaurant_1,Royal Kitchen Family Restaurant,755.233125,16
2,Restaurant_2,Royal Delights South Indian,741.113684,19
4,Restaurant_4,Annapurna Curry House Pure Veg,664.400000,17
6,Restaurant_6,Ruchi Restaurant South Indian,718.878125,16
7,Restaurant_7,Ruchi Biryani Punjabi,1002.140625,16
...,...,...,...,...
488,Restaurant_488,New Biryani North Indian,872.171053,19
489,Restaurant_489,Swagath Foods Punjabi,875.268333,18
495,Restaurant_495,Ruchi Curry House North Indian,782.515556,18


In [28]:
mcq_restaurants = ["Grand Cafe Punjabi","Grand Restaurant South Indian","Ruchi Mess Multicuisine","Ruchi Foods Chinese"]

In [29]:
filtered_restaurants.reset_index()[filtered_restaurants.reset_index()["restaurant_name_order"].isin(mcq_restaurants)].sort_values(by="avg_order_value", ascending=False)

,restaurant_id,restaurant_name_master,restaurant_name_order,avg_order_value,total_orders
188,385,Restaurant_385,Ruchi Mess Multicuisine,938.202778,18
49,116,Restaurant_116,Grand Cafe Punjabi,775.850000,12
205,421,Restaurant_421,Ruchi Foods Chinese,686.603158,19


In [31]:
combo_revenue = (df.groupby(["membership", "cuisine"])["total_amount"].sum().sort_values(ascending=False))
combo_revenue

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [47]:
o = [("Gold", "Indian"),("Gold", "Italian"),("Regular", "Indian"),("Regular", "Chinese")]

combo_revenue.loc[o].sort_values(ascending=False)

membership  cuisine
Gold        Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [33]:
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

In [34]:
df["quarter"] = df["order_date"].dt.to_period("Q")

In [35]:
quarter_revenue = (df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
)
quarter_revenue

quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [36]:
gold_orders_count = df[df["membership"] == "Gold"]["order_id"].nunique()
gold_orders_count

4987

In [37]:
hyderabad_revenue = round(df[df["city"] == "Hyderabad"]["total_amount"].sum()
)
hyderabad_revenue

1889367

In [38]:
distinct_users = df["user_id"].nunique()
distinct_users

2883

In [39]:
gold_aov = round(df[df["membership"] == "Gold"]["total_amount"].mean(),2)
gold_aov

np.float64(797.15)

In [40]:
high_rating_orders = df[df["rating"] >= 4.5]["order_id"].nunique()
high_rating_orders

3374

In [41]:
gold_city_revenue = (df[df["membership"] == "Gold"].groupby("city")["total_amount"].sum())
top_gold_city = gold_city_revenue.idxmax()
top_gold_city

'Chennai'

In [42]:
orders_in_top_city = df[(df["membership"] == "Gold") &(df["city"] == top_gold_city)]["order_id"].nunique()
orders_in_top_city

1337

In [43]:
df.shape[0]

10000